# Chicago Weather Forecasting: Model Experimentation

In this notebook I shall explain how one can build models for weather forecasting and offer evalutation of these models.

## Problem Scope Definition

Before we proceed with modelling, we need to define what the scope of this project is.

### _(1) What we are forecasting_

The goal is to predict a number of weather characteristics in the future for the city of Chicago. We shall select the basic componenets of a weather situation that a layperson will be considering for planning purposes. Think about deciding whether to go on a Sunday picnic on the beach or go hiking.  

With that in mind, the following quantities are chosen:

- Temperature (in Farenheit degrees)
- Wind Speed (in miles per hour) 
- Precipitation (whether or not there will be rain / snow or hail)
- Clarity (whether or not the sky will be free of clouds)

Of those, forecasting Temperature and Wind are **Regression** problems, while forecasting Precipitation or Clarity are **Binary Classification** problems.

We shall attempt to predict weather for the following durations in advance:

- 6 hours
- 12 hours
- 18 hours
- 24 hours

(Preliminary experimentation has proven longer term forecasts to be not feasible with the data available)

### _(2)  Model Dataset_

As explained previously, we shall be relying on US Government (NOAA) datasets containing **hourly** weather reports for the weather station in Chicago as well as nearby stations in the US Midwest. 

In this notebook we shall work with data for 10 years, 2011-2020 (inclusively) from the following locations:

- Chicago, IL (target location)
- Cedar Rapids, IA
- Des Moines, IA
- Rochester, MN
- Quincy, IL
- Madison, WI
- St Louis, MO
- Green Bay, WI
- Lansing, MI

Most of these locations are **West** of Chicago as we previously determined through correlation analysis that locations in that direction have much more effect on weather in Chicago than locations in other directions.

Finally, we shall be using preprocessed reports rather than the non-intuitive raw NOAA reports. See the following  (Timestamp is followed by the 4 quantities we aim to forecast as well as a few more for demo purposes):

In [93]:
import pandas as pd

df = pd.read_csv('../processed-data/noaa_2011-2020_chicago_PREPROC.csv')
subset_df = df [['DATE', 'Temp', 'WindSpeed', '_is_precip', '_is_clear', 'CloudCondition', 'WeatherType', 
                 'Pressure', 'Humidity', '_wind_dir_sin', '_wind_dir_cos']]
subset_df.head(20)

,DATE,Temp,WindSpeed,_is_precip,_is_clear,CloudCondition,WeatherType,Pressure,Humidity,_wind_dir_sin,_wind_dir_cos
0,2011-01-01 00:00:00,40.333333,13.0,0,0,Cloudy,NoPrecipitation,29.720,71.333333,-0.939693,-3.420201e-01
1,2011-01-01 01:00:00,37.000000,17.0,0,0,Cloudy,NoPrecipitation,29.735,70.000000,-0.984808,-1.736482e-01
2,2011-01-01 02:00:00,36.000000,17.0,0,0,Cloudy,NoPrecipitation,29.750,70.000000,-0.866025,-5.000000e-01
3,2011-01-01 03:00:00,32.000000,15.0,0,0,MostlyCloudy,NoPrecipitation,29.750,61.000000,-0.866025,-5.000000e-01
4,2011-01-01 04:00:00,31.000000,16.0,0,1,PartlyCloudy,NoPrecipitation,29.760,61.000000,-0.866025,-5.000000e-01
5,2011-01-01 05:00:00,28.000000,18.0,0,1,MostlyClear,NoPrecipitation,29.770,63.000000,-0.866025,-5.000000e-01
6,2011-01-01 06:00:00,27.500000,17.0,0,0,MostlyCloudy,NoPrecipitation,29.785,67.500000,-0.866025,-5.000000e-01
7,2011-01-01 07:00:00,25.000000,20.0,0,0,MostlyCloudy,NoPrecipitation,29.810,75.000000,-0.866025,-5.000000e-01
8,2011-01-01 08:00:00,23.000000,21.0,0,0,Cloudy,NoPrecipitation,29.870,65.000000,-0.866025,-5.000000e-01
9,2011-01-01 09:00:00,21.000000,23.0,0,0,Cloudy,NoPrecipitation,29.890,62.000000,-0.939693,-3.420201e-01


### _(3)  Aggregated Forecasting_

As explained previously, the datasets are chronological lists of hourly data points. What does it mean to forecast each of the target quantities, say, 12h, in advance?

Predicting weather for a particular hour may not serve us particularly well. Consider the following situations: 

- let's say it is 11PM and we are considering a picnic at 11AM the next day. If it does not rain at 11AM but it does rain at 10AM or 1PM, the picnic is a bad idea.

- similarly, if we are considering kayaking, if the wind is going to be 5mph at 11AM but 30mph at 2PM, we should reconsider

*To address such concerns, we shall be attempting to forecast not weather for the exact target hour but rather some kind of **aggregation over an interval** centered around that hour.*

In the code to follow we shall rely on something called **Aggregation Half Interval (AHI)**. For example, if AHI = 3 and the target hour is 11AM, we shall be considering the interval spanning 08AM to 02PM. 

Let us now come up with the definitions for what that means for each of the 4 forecasted quantities:

| Quantity | AHI | Aggregation Rule |
| --- | --- | --- |
| Temperature | 1h | Average |
| WindSpeed   | 2h | Average |
| Precipitation | 3h | True if any element is True |
| Clarity | 3h  | True if all elements are True |

The first two rows for analog quantities are self explanatory: we are smoothing the prediction over an interval by averaging. Temperature has a smaller interval as it is much more directly dependent on time of day than wind.

The last two rows for binary quantities say that if *any* hour during the interval is Rainy or Not Fair, the resulting forecast too is Rainy or Not Fair. As per the situation described above, if it rains anywhere close to the hour for which we are forecasting, we'll get wet. Similarly, if it is not fair anywhere close to that hour, our sun tanning won't go well. 

## Preparing the Learning Data Set

### (1) _Merge Data from All Locations_

We need to do a JOIN on all the weather reports whose data we'll be feeding into our models. The following functions perform the merge and drop any irrelevant columns:

In [94]:
def buildFeatureSet(targetLocationFile, adjacentLocationFiles, predictedVariable, featuresToUse):
    target_df = pd.read_csv(targetLocationFile, parse_dates=['DATE'])
    target_df = dropUnusedColumns(target_df, predictedVariable, featuresToUse)
    merged_df = target_df
    suffix_no = 1

    # Merge adjacent location files one by one relying on DATE
    for adjacentLocationFile in adjacentLocationFiles:
        adjacent_df = pd.read_csv(adjacentLocationFile, parse_dates=['DATE'])
        adjacent_df = dropUnusedColumns(adjacent_df, predictedVariable, featuresToUse)

        #Take control of column name suffix in the dataset being merged in
        adjacent_df = adjacent_df.add_suffix(str(suffix_no))
        adjacent_df = adjacent_df.rename(columns = {"DATE{}".format(suffix_no) :'DATE'})
        merged_df = pd.merge(merged_df, adjacent_df, on='DATE')
        suffix_no = suffix_no + 1

    # DATE column is of no use in the modelling stage (we only needed it for merging)
    merged_df = merged_df.drop(columns=['DATE'])
    return merged_df

#======================================================================
# Keep only the DATE column, the variable we are predicting and the variables that we use for prediction
def dropUnusedColumns(df, predictedVariable, featuresToUse):
    all_columns = featuresToUse.copy()
    all_columns.append('DATE')
    all_columns.append(predictedVariable)
    df = df[all_columns]

    return df


Quick illustration:

In [95]:
featureset = buildFeatureSet(
    '../processed-data/noaa_2011-2020_chicago_PREPROC.csv',
    ['../processed-data/noaa_2011-2020_cedar-rapids_PREPROC.csv', 
         '../processed-data/noaa_2011-2020_des-moines_PREPROC.csv'],
    predictedVariable='WindSpeed',
    featuresToUse = ['_wind_dir_sin', '_wind_dir_cos']
    )
featureset.head()

,_wind_dir_sin,_wind_dir_cos,WindSpeed,_wind_dir_sin1,_wind_dir_cos1,WindSpeed1,_wind_dir_sin2,_wind_dir_cos2,WindSpeed2
0,-0.939693,-0.342020,13.0,-0.802123,-0.597159,23.666667,-0.866025,-0.500000,23.5
1,-0.984808,-0.173648,17.0,-0.642788,-0.766044,25.000000,-0.939693,-0.342020,24.0
2,-0.866025,-0.500000,17.0,-0.766044,-0.642788,23.000000,-0.984808,-0.173648,22.0
3,-0.866025,-0.500000,15.0,-0.866025,-0.500000,23.000000,-0.939693,-0.342020,22.0
4,-0.866025,-0.500000,16.0,-0.866025,-0.500000,23.000000,-0.939693,-0.342020,16.0


We have a set of 3 variables of interest: `WindSpeed` (predicted) as well as `_wind_dir_sin` and `_wind_dir_cos` (to be used for predicting). As you can see, the dataset above has these variables repeated 3 times, once for each location. This merged kind of dataset will be used going forward.

### (2) _Split and Normalize the Data_

Before we can train models we must split the data into the 3 subsets:

- *Training*: the actual data that we'll be training on. This is the largest subset.
- *Validation*: the dataset to be used for model tuning during training to check the model periodically
- *Testing*: the dataset that will be hidden from the model training process and be used for final model evaluation

Of course, we'll also need to normalize the features on which we are training to avoid algorithm execution issues like gradient explosion. The following code achieves both:

In [96]:
import warnings
warnings.filterwarnings('ignore')

def normalizeData(trainDf, valDf,  testDf, predictedVariable, featuresToUse, adjacentLocationCount):

    columns_to_normalize = featuresToUse.copy()

    prefixes_to_normalize = featuresToUse.copy()
    prefixes_to_normalize.append(predictedVariable)
    for loc in range(1, 1 + adjacentLocationCount):
        for prefix in prefixes_to_normalize:
            columns_to_normalize.append("{}{}".format(prefix, loc))

    # Normalize input data but not the target variable
    train_mean = trainDf[columns_to_normalize].mean()
    train_std = trainDf[columns_to_normalize].std()

    trainDf[columns_to_normalize] = (trainDf[columns_to_normalize] - train_mean) / train_std
    valDf[columns_to_normalize] = (valDf[columns_to_normalize] - train_mean) / train_std
    testDf[columns_to_normalize] = (testDf[columns_to_normalize] - train_mean) / train_std

    return trainDf, valDf, testDf


# Split the data: 6 years for training, 2 for validation & 2 for testing
n = len(featureset)
train_df = featureset[0 : int(n*0.60)]
val_df = featureset[int(n*0.60) : int(n*0.80)]
test_df = featureset[int(n*0.80) : ]

# Normalize input data
train_df, val_df, test_df = normalizeData(train_df, val_df, test_df, 
                                          'WindSpeed', ['_wind_dir_sin', '_wind_dir_cos'], 2)

train_df.head()


,_wind_dir_sin,_wind_dir_cos,WindSpeed,_wind_dir_sin1,_wind_dir_cos1,WindSpeed1,_wind_dir_sin2,_wind_dir_cos2,WindSpeed2
0,-1.188696,-0.466556,13.0,-1.144549,-0.762454,2.369524,-1.332160,-0.567817,2.530257
1,-1.256421,-0.236236,17.0,-0.902651,-0.987495,2.595942,-1.448306,-0.362728,2.621978
2,-1.078108,-0.682661,17.0,-1.089776,-0.823255,2.256315,-1.519435,-0.144148,2.255092
3,-1.078108,-0.682661,15.0,-1.241564,-0.632990,2.256315,-1.448306,-0.362728,2.255092
4,-1.078108,-0.682661,16.0,-1.241564,-0.632990,2.256315,-1.448306,-0.362728,1.154431


### (3) _Prepare the Data for TensorFlow_

We still have further to go before we can use TensorFlow to build models. 

First, we need to create a Sliding Window type data structure containing a number of observations in the Past. For example if we are forecasting _Temperature_ 12h in advance and we want to look back 3 hours, we need `Temperature[-12h], Temperature[-13h], Temperature[-14h]` all in one row.

Second, TensorFlow is quite particular about what form the input data should take:

_Typically data in TensorFlow is packed into arrays where the outermost index is across examples (the "batch" dimension). The middle indices are the "time" or "space" (width, height) dimension(s). The innermost indices are the features_ (see https://www.tensorflow.org/tutorials/structured_data/time_series).

The following class borrowed from the manual above takes the Pandas dataset and massages it into a Sliding Window tensor of the correct shape:

In [97]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

class WindowGenerator():

    def __init__(self, 
        input_width, # Lookback Window (hours into the past to base predictions on)
        label_width, # Aggregation Interval (how many hours of data we'll be predicting)
        shift, # How many hours in advance we'll be predicting
        train_df, val_df, test_df, # Training, Validation and Testing sets
        label_columns=None):

        # Store the raw data.
        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
          self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
          data=data,
          targets=None,
          sequence_length=self.total_window_size,
          sequence_stride=1,
          shuffle=False,
          batch_size=32,)

        ds = ds.map(self.split_window)

        return ds

    @property
    def train(self):
        return self.make_dataset(self.train_df)

    @property
    def val(self):
        return self.make_dataset(self.val_df)

    @property
    def test(self):
        return self.make_dataset(self.test_df)

    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result

We shall now use the above to demonstrate generation of Tensorflow Datasets: 

In [98]:
wg = WindowGenerator(
    input_width = 12, # Take 12h of history into account
    label_width = 5,  # Corresponds to aggreagation half-interval of 2h
    shift = 4, # Forecast 6 hours in Advance (6 - (AHI=2) = 4)
    label_columns=['WindSpeed'],
    train_df = train_df, val_df = val_df, test_df = test_df # Create Tensorflow datasets for all 3 subsets
)

for example_inputs, example_labels in wg.train.take(1):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')

Inputs shape (batch, time, features): (32, 12, 9)
Labels shape (batch, time, features): (32, 5, 1)


Those shapes are explained as follows:

- 32 is the batch size (Tensorflow trains on data in batches)
- 12 is the number of hours: for Inputs it is 12 because we are looking 12 hours back while for Outputs it is 5 because we are aggregating over 5h 
- 9 is the number of input features: `WindSpeed`, `_wind_dir_sin` and `_wind_dir_cos` for each of the 3 locations

## Working With Models

### (1) _Our First Model: Linear_

It is time to build our very first model! It is the simplest one that there is, the `Linear` kind. The approach that we'll take is also borrowed from https://www.tensorflow.org/tutorials/structured_data/time_series . The goal is to predict *all* target labels in the aggregation interval in one shot. So in the case above, where the interval is 5, we'll be predicting 5 values in chronological order. 

We are going to make our model creation and evaluation code work for both Binary Classification and Regression problems as follows:

In [106]:
REGRESSION_METRICS = [ tf.keras.metrics.RootMeanSquaredError()]
CLASSIFICATION_METRICS =[tf.keras.metrics.Recall(name="recall"), tf.keras.metrics.Precision()]
def getActivationLossAndMetrics(isBinary):
    activation, loss, metrics = "linear", 'mean_squared_error', REGRESSION_METRICS
    if isBinary:
        activation, loss, metrics = "sigmoid", tf.keras.losses.BinaryCrossentropy(), CLASSIFICATION_METRICS

    return activation, loss, metrics


def buildLinearModel(isBinary, label_width):
    _activation, _loss, _metrics = getActivationLossAndMetrics(isBinary)
    model = tf.keras.Sequential([
        # Use all time steps
        tf.keras.layers.Flatten(),

        # Note that there are no hidden layers, only the output one, which makes the model linear.
        tf.keras.layers.Dense(units=label_width, activation = _activation, kernel_initializer=tf.initializers.zeros()),
        tf.keras.layers.Reshape([label_width, 1]),
    ])
    model.compile(loss=_loss, optimizer='adam', metrics = _metrics)
    return model

### (2) _Model Evaluation_

Before we proceed to actually train our model we need to establish the criteria based on which we'll evaluate it. 

This is a Regression model so we'll use the following metrics:

- Root Mean Squared Error
- Absolute Mean Error (less affected by outliers)
- R2 Score
- MAPE (Mean Absolute Percentage Error)

While we are on the subject, we shall also define the Binary Classification Metrics (we'll use them later):

- Recall
- Precision
- F1 Score
- MCC (Matthew Coefficient, arguably the most balanced measure of Binary Classification performance)

Finally, we'll need some extra code to evaluate the model. Remember, our models are predicting a sequence of values over a time interval, whereas our application calls for aggregating values over that interval. The code below will handle that as well by parsing through the Tensors returned by the model. 

In [100]:
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
import math

# https://www.statology.org/mape-python/
def calcMape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred) 
    actual[abs(actual )< 0.1] = 0.1 # A meh hack to avoid division by 0

    return np.mean(np.abs((actual - pred) / actual )) * 100

# https://kodify.net/python/math/truncate-decimals/	
def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

def evaluateClassificationModel(model, testSet, options = []):

    predicted_labels =(model.predict(testSet, verbose = 1) > 0.5).astype("int32")
    true_labels = np.concatenate([y for x, y in testSet], axis=0)

    assert len(predicted_labels) == len (true_labels)

    # We are forecasting for a number of hours: 
    # aggregate each forecast series using the "True iff 1 or more is True" rule (default) or
    # "True iff all True" (option specified)
    predicted_agg = []
    true_agg = []
    for i in range(0, len(predicted_labels)):
        predicted_i = predicted_labels[i].flatten()
        true_i = true_labels[i].flatten()

        if not "TrueIfAllTrue" in options:
            predicted_i_agg = 1 if sum(predicted_i) > 0 else 0
            true_i_agg = 1 if sum(true_i) > 0 else 0
        else:
            predicted_i_agg = 1 if sum(predicted_i) == len(predicted_i) else 0
            true_i_agg = 1 if sum(true_i) == len(true_i) else 0

        predicted_agg.append(predicted_i_agg)
        true_agg.append(true_i_agg)

    recall = truncate(recall_score(true_agg, predicted_agg), 2)
    precision = truncate(precision_score(true_agg, predicted_agg), 2)
    f1 = truncate(f1_score(true_agg, predicted_agg), 2)
    mcc = truncate(matthews_corrcoef(true_agg, predicted_agg), 2)

    print(confusion_matrix(true_agg, predicted_agg))
    print("Recall = {}, Precision = {}, F1 = {}, MCC = {}".format(recall, precision, f1, mcc))

    return {
        "Recall": recall,
        "Precision": precision, 
        "F1:" : f1,
        "MCC:": mcc
    }	

def evaluateRegressionModel(model, testSet, options=[]):
    predicted_values = model.predict(testSet)
    true_values = np.concatenate([y for x, y in testSet], axis=0)

    assert len(predicted_values) == len (true_values)

    predicted_agg = []
    true_agg = []
    for i in range(0, len(predicted_values)):
        predicted_i = predicted_values[i].flatten()
        true_i = true_values[i].flatten()

        predicted_i_agg, true_i_agg = np.mean(predicted_i), np.mean(true_i)
        predicted_agg.append(predicted_i_agg)
        true_agg.append(true_i_agg)

    rmse = truncate(math.sqrt(mean_squared_error(true_agg, predicted_agg)), 2)
    mae = truncate(mean_absolute_error(true_agg, predicted_agg), 2)
    r2 = truncate(r2_score(true_agg, predicted_agg), 2)
    mape = truncate(calcMape(true_agg, predicted_agg), 2) 

    print("R2 = {}, RMSE = {}, MAE = {}, MAPE = {}%".format(r2, rmse, mae, mape))
    return {
        'R2' : r2,
        'RMSE' : rmse,
        'MAE' : mae,
        'MAPE' : "{}%".format(mape)
    }



### (3) _A Note on Avoiding Overfitting_

Overfitting is a common problem in Machine Learning, especially with Neural Networks. Given enough model complexity and iteration epochs, learing algorithms tend to learn the Training Dataset perfectly but generalize poorly to previously unseen (Test/Validation) data. 

We shall use the `EarlyStopping` Callback mechanism provided by TF. The idea is that once the target metric stops improving on the Validation Dataset we stop the training. We then go with the learned model weights  for which that target metic is optimal on the Validation Dataset. This is the simplest regularization technique out there and for the purposes of this exercise it has proven to be adequate. 

For Regression problems we shall use `RMSE` as the target metric and for Binary Classification problems we shall use `Recall`. The idea behind the latter is that both Precipitation and Clarity are much less common than their opposites (Chicago is not exactly known for wonderful weather) and we shall stir the model towards avoiding False Negatives. That is, we'll try to catch as many Rainy / Clear days as we can worrying less about mistakenly flagging not rainy and not clear days.

Of course, Overfitting is less of a concern for the Linear model that we'll be demonstrating first.

### (4) _Fit our First Model and Evaluate the Result_

We are now ready to fit our first model to try and predict Temperature. 

Note that we'll configure Early Stopping to prevent us from Overfitting the Training Set as well as save on training time when the returns become too deminishing. See the above section for explainations.

First, though, build a less skimpy featureset than what we demoed above to get better model performance.

In [101]:
# Repeat the Previous steps for a better dataset
features_to_use = ['_day_sin', '_day_cos', '_hour_sin', '_hour_cos', 'DewPoint', 'WindSpeed', '_cloud_intensity']
featureset = buildFeatureSet(
    '../processed-data/noaa_2011-2020_chicago_PREPROC.csv',
    ['../processed-data/noaa_2011-2020_cedar-rapids_PREPROC.csv', 
     '../processed-data/noaa_2011-2020_madison_PREPROC.csv',
     '../processed-data/noaa_2011-2020_des-moines_PREPROC.csv',
    '../processed-data/noaa_2011-2020_rochester_PREPROC.csv'],
    predictedVariable='Temp',
    featuresToUse = features_to_use
    )
n = len(featureset)
train_df = featureset[0 : int(n*0.60)]
val_df = featureset[int(n*0.60) : int(n*0.80)]
test_df = featureset[int(n*0.80) : ]
train_df, val_df, test_df = normalizeData(train_df, val_df, test_df, 
                                          'Temp', features_to_use, 4)
window = WindowGenerator(
    input_width = 2, # Take 2h of history into account
    label_width = 3,  # Corresponds to aggreagation half-interval of 1h
    shift = 5, # Forecast 6 hours in Advance (6 - (AHI=1) = 5)
    label_columns=['Temp'],
    train_df = train_df, val_df = val_df, test_df = test_df # Create Tensorflow datasets for all 3 subsets
)


Now use that larger dataset to do the modelling:

In [102]:
# Build the Model
model = buildLinearModel(isBinary = False, label_width = 3)

# Configure early stopping so we don't learn the training data too well at the expense of test/validation data (overfit)
esCallback = tf.keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error', mode="min", patience=5, min_delta = 0.1, restore_best_weights = True)

# NOTE: provide the Validation Dataset so that the Model does not check itself on Training Data
model.fit(window.train, validation_data = window.val, callbacks = [esCallback], epochs = 20)

print("\r\n- Performance on *TRAINING* data:")
evaluateRegressionModel(model, window.train)
print("\r\n- Performance on *VALIDATION* data:")
evaluateRegressionModel(model, window.val)
print("\r\n- Performance on *TEST* data:")
evaluateRegressionModel(model, window.test)

Epoch 1/20
1644/1644 [==============================] - 1s 687us/step - loss: 629.5239 - root_mean_squared_error: 24.2431 - val_loss: 19.4044 - val_root_mean_squared_error: 4.4050
Epoch 2/20
1644/1644 [==============================] - 1s 706us/step - loss: 19.6446 - root_mean_squared_error: 4.4304 - val_loss: 18.1651 - val_root_mean_squared_error: 4.2621
Epoch 3/20
1644/1644 [==============================] - 1s 764us/step - loss: 19.0656 - root_mean_squared_error: 4.3645 - val_loss: 17.6788 - val_root_mean_squared_error: 4.2046
Epoch 4/20
1644/1644 [==============================] - 1s 697us/step - loss: 18.5970 - root_mean_squared_error: 4.3104 - val_loss: 17.1504 - val_root_mean_squared_error: 4.1413
Epoch 5/20
1644/1644 [==============================] - 1s 710us/step - loss: 18.0705 - root_mean_squared_error: 4.2488 - val_loss: 16.6797 - val_root_mean_squared_error: 4.0841
Epoch 6/20
1644/1644 [==============================] - 1s 772us/step - loss: 17.5628 - root_mean_squared_er

{'R2': 0.96, 'RMSE': 3.77, 'MAE': 2.9, 'MAPE': '9.3%'}

Not bad! Our Linear Model for predicting temperature 6 hours in advance is, on average, only `3F` off (RMSE is closer to `4F` as it is more sensitive to outliers by design).

As a sanity check let's see how we do trying to predict Tempearature 24 hours in advance:

In [103]:
window = WindowGenerator(
    input_width = 2, # Take 2h of history into account
    label_width = 3,  # Corresponds to aggreagation half-interval of 1h
    shift = 23, # Forecast 6 hours in Advance (24 - (AHI=1) = 23)
    label_columns=['Temp'],
    train_df = train_df, val_df = val_df, test_df = test_df # Create Tensorflow datasets for all 3 subsets
)

# Build the Model
model = buildLinearModel(isBinary = False, label_width = 3)

# Configure early stopping so we don't learn the training data too well at the expense of test/validation data (overfit)
esCallback = tf.keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error', mode="min", patience=5, min_delta = 0.1, restore_best_weights = True)

# NOTE: provide the Validation Dataset so that the Model does not check itself on Training Data
model.fit(window.train, validation_data = window.val, callbacks = [esCallback], epochs = 20)

print("\r\n- Performance on *TRAINING* data:")
evaluateRegressionModel(model, window.train)
print("\r\n- Performance on *VALIDATION* data:")
evaluateRegressionModel(model, window.val)
print("\r\n- Performance on *TEST* data:")
evaluateRegressionModel(model, window.test)

Epoch 1/20
1644/1644 [==============================] - 1s 727us/step - loss: 651.4777 - root_mean_squared_error: 17.3271 - val_loss: 62.9535 - val_root_mean_squared_error: 7.9343
Epoch 2/20
1644/1644 [==============================] - 1s 698us/step - loss: 62.7116 - root_mean_squared_error: 7.9154 - val_loss: 58.8930 - val_root_mean_squared_error: 7.6742
Epoch 3/20
1644/1644 [==============================] - 1s 713us/step - loss: 59.7334 - root_mean_squared_error: 7.7249 - val_loss: 57.0960 - val_root_mean_squared_error: 7.5562
Epoch 4/20
1644/1644 [==============================] - 1s 712us/step - loss: 57.7763 - root_mean_squared_error: 7.5974 - val_loss: 55.6181 - val_root_mean_squared_error: 7.4578
Epoch 5/20
1644/1644 [==============================] - 1s 732us/step - loss: 56.1512 - root_mean_squared_error: 7.4898 - val_loss: 54.4145 - val_root_mean_squared_error: 7.3766
Epoch 6/20
1644/1644 [==============================] - 1s 743us/step - loss: 54.8301 - root_mean_squared_er

{'R2': 0.87, 'RMSE': 7.23, 'MAE': 5.45, 'MAPE': '21.87%'}

As we expected, once we increased prediction time by 18h the quality of our prediction went down. Our average error grew from under `3F` to over `5F`. Still, the forecast is quite good as errors of this magnitude allow us to know how to dress (more or less).

Before we move on to other, more interesting, models let's try to fit a Binary Classification problem. Specifically, we'll fit `Precipitation` 12 hours in advance.

In [107]:
features_to_use = ['_day_sin', '_day_cos', '_hour_sin', '_hour_cos', 'DewPoint', 'WindSpeed', '_cloud_intensity']
featureset = buildFeatureSet(
    '../processed-data/noaa_2011-2020_chicago_PREPROC.csv',
    ['../processed-data/noaa_2011-2020_cedar-rapids_PREPROC.csv', 
     '../processed-data/noaa_2011-2020_madison_PREPROC.csv',
     '../processed-data/noaa_2011-2020_des-moines_PREPROC.csv',
    '../processed-data/noaa_2011-2020_rochester_PREPROC.csv'],
    predictedVariable='_is_precip',
    featuresToUse = features_to_use
    )

# Split
n = len(featureset)
train_df = featureset[0 : int(n*0.60)]
val_df = featureset[int(n*0.60) : int(n*0.80)]
test_df = featureset[int(n*0.80) : ]

# Normalize
train_df, val_df, test_df = normalizeData(train_df, val_df, test_df, 
                                          '_is_precip', features_to_use, 4)

window = WindowGenerator(
    input_width = 2, # Take 2h of history into account
    label_width = 7,  # Corresponds to aggreagation half-interval of 3h
    shift = 9, # Forecast 12 hours in Advance (12 - (AHI=3) = 9)
    label_columns=['_is_precip'],
    train_df = train_df, val_df = val_df, test_df = test_df # Create Tensorflow datasets for all 3 subsets
)

# Build the Model
model = buildLinearModel(isBinary = True, label_width = 7)

# Configure early stopping so we don't learn the training data too well at the expense of test/validation data (overfit)
esCallback = tf.keras.callbacks.EarlyStopping(monitor='val_recall', mode="max", patience=5, min_delta = 0.05, restore_best_weights = True)

# NOTE: provide the Validation Dataset so that the Model does not check itself on Training Data
model.fit(window.train, validation_data = window.val, callbacks = [esCallback], epochs = 20)

print("\r\n- Performance on *TRAINING* data:")
evaluateClassificationModel(model, window.train)
print("\r\n- Performance on *VALIDATION* data:")
evaluateClassificationModel(model, window.val)
print("\r\n- Performance on *TEST* data:")
evaluateClassificationModel(model, window.test)

Epoch 1/20
1644/1644 [==============================] - 2s 904us/step - loss: 0.5485 - recall: 0.4581 - precision_12: 0.2761 - val_loss: 0.3535 - val_recall: 0.3887 - val_precision_12: 0.5149
Epoch 2/20
1644/1644 [==============================] - 1s 878us/step - loss: 0.3414 - recall: 0.3918 - precision_12: 0.5367 - val_loss: 0.2910 - val_recall: 0.2994 - val_precision_12: 0.5923
Epoch 3/20
1644/1644 [==============================] - 1s 879us/step - loss: 0.2897 - recall: 0.3294 - precision_12: 0.6083 - val_loss: 0.2727 - val_recall: 0.2874 - val_precision_12: 0.6317
Epoch 4/20
1644/1644 [==============================] - 1s 805us/step - loss: 0.2737 - recall: 0.3230 - precision_12: 0.6366 - val_loss: 0.2664 - val_recall: 0.2917 - val_precision_12: 0.6404
Epoch 5/20
1644/1644 [==============================] - 1s 861us/step - loss: 0.2680 - recall: 0.3270 - precision_12: 0.6448 - val_loss: 0.2640 - val_recall: 0.2971 - val_precision_12: 0.6397
Epoch 6/20
1644/1644 [==================

{'Recall': 0.36, 'Precision': 0.76, 'F1:': 0.49, 'MCC:': 0.43}

Notice that `val_recall` starts going up after the 3d iteration while the `recall` (computed on the Training Set) keeps going down. This reassures us that Early Stopping is an effective strategy in preventing Overfitting. 

Overall, this isn't a great model. We correctly catch just over a third of rainy / snowy periods. We may benefit from more complex models going forward. 

### (5) _Non-Linear Models_


We shall explore the following models that are non-Linear in the hopes that they'll give us better results. These models shall be:

- Single Hidden Layer Neural Network
- Two Hidden Layer Neural Network
- Convolutional Neural Network with a single Convolutional Layer

(_NOTE: LSTM/RNN too was considered but through some preliminary experimentation I concluded that the **order** of datapoints does not noticeably affect the forecast. Since this type of network thrives on order being meanigful to the variable being learned, I did not deem RNN/LSTM worthy of inclusion due to their long training times_)

The following functions construct each of these 3 additional kinds

In [118]:
# Single Hidden Layer
def buildSimpleNNModel(isBinary, label_width):
    _activation, _loss, _metrics  = getActivationLossAndMetrics(isBinary)

    model = tf.keras.Sequential([
        # Use all time steps
        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(units=400, activation='relu'),
        tf.keras.layers.Dense(units=label_width, activation = _activation, kernel_initializer=tf.initializers.zeros()),

        # Add back the time dimension.
        # Shape: (outputs) => (1, outputs)
        tf.keras.layers.Reshape([label_width, 1]),
    ])
    model.compile(loss=_loss, optimizer='adam', metrics = _metrics)
    return model

# Two Hidden Layers
def buildDNNModel(isBinary, label_width):
    _activation, _loss, _metrics = getActivationLossAndMetrics(isBinary)

    model = tf.keras.Sequential([
         # Use all time steps
        tf.keras.layers.Flatten(),

        tf.keras.layers.Dense(units=400, activation='relu'),
        tf.keras.layers.Dense(units=200, activation='relu'),

        tf.keras.layers.Dense(units=label_width, activation = _activation, kernel_initializer=tf.initializers.zeros()),

        # Add back the time dimension.
        # Shape: (outputs) => (1, outputs)
        tf.keras.layers.Reshape([label_width, 1]),
    ])
    model.compile(loss=_loss, optimizer='adam', metrics = _metrics)
    return model

# Convolutional
def buildConvModel(isBinary, lookbackHours, label_width):
    _activation, _loss, _metrics = getActivationLossAndMetrics(isBinary)
    CONV_WIDTH = 4

    model = tf.keras.Sequential([
        # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
        tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),

        tf.keras.layers.Conv1D(filters=200,
                               kernel_size=(CONV_WIDTH),
                               activation='relu'),
        #tf.keras.layers.Dense(units=100, activation='relu'),
        tf.keras.layers.Dense(units=label_width, activation=_activation, kernel_initializer=tf.initializers.zeros()),
        tf.keras.layers.Reshape([label_width, 1]),
        ])

    model.compile(loss=_loss, optimizer='adam', metrics = [_metrics])
    return model

Let us now repeat the above experiment (predict `Precipitation` 12 hours in advance) using the second of the additional networks above:

In [117]:
# Build the Model
model = buildDNNModel(isBinary = True, label_width = 7)

# Configure early stopping so we don't learn the training data too well at the expense of test/validation data (overfit)
esCallback = tf.keras.callbacks.EarlyStopping(monitor='val_recall', mode="max", patience=10, min_delta = 0.001, restore_best_weights = True)

# NOTE: provide the Validation Dataset so that the Model does not check itself on Training Data
model.fit(window.train, validation_data = window.val, callbacks = [esCallback], epochs = 20)

print("\r\n- Performance on *TRAINING* data:")
evaluateClassificationModel(model, window.train)
print("\r\n- Performance on *VALIDATION* data:")
evaluateClassificationModel(model, window.val)
print("\r\n- Performance on *TEST* data:")
evaluateClassificationModel(model, window.test)

Epoch 1/20
1644/1644 [==============================] - 3s 1ms/step - loss: 0.3062 - recall: 0.2655 - precision_12: 0.6130 - val_loss: 0.2657 - val_recall: 0.2948 - val_precision_12: 0.6126
Epoch 2/20
1644/1644 [==============================] - 2s 1ms/step - loss: 0.2714 - recall: 0.3388 - precision_12: 0.6354 - val_loss: 0.2660 - val_recall: 0.3258 - val_precision_12: 0.6131
Epoch 3/20
1644/1644 [==============================] - 2s 1ms/step - loss: 0.2675 - recall: 0.3430 - precision_12: 0.6441 - val_loss: 0.2655 - val_recall: 0.3213 - val_precision_12: 0.6168
Epoch 4/20
1644/1644 [==============================] - 2s 1ms/step - loss: 0.2653 - recall: 0.3453 - precision_12: 0.6518 - val_loss: 0.2639 - val_recall: 0.3198 - val_precision_12: 0.6216
Epoch 5/20
1644/1644 [==============================] - 2s 1ms/step - loss: 0.2633 - recall: 0.3492 - precision_12: 0.6565 - val_loss: 0.2659 - val_recall: 0.3145 - val_precision_12: 0.6219
Epoch 6/20
1644/1644 [============================

{'Recall': 0.4, 'Precision': 0.71, 'F1:': 0.51, 'MCC:': 0.43}

That is noticeably better than the Linear model we tried for the same problem just above! 

#  Conclusion: Compare and Choose the Models

We have presented four kinds of different models. The following script automatically builds and evaluates all of those models for all predicted variables and for all prediction intervals: https://github.com/sergeypine/WeatherLearner/blob/capstone-model-experiments/model-prototyping/NOAA_learner_benchmark2.py

We shall not go over it here as it repackages the same code that was already presented above. The whole execution takes over an hour so reproducing that here wouldn't be efficient either. In total, 64 executions are performed (`4 models x 4 predicted quantities x 4 prediction intervals`)

The tables below summarize the results based on the target metric discussed above. The "winners" are **highlighted**. Note that if two models give roughly the same results, the simpler one is chosen.

(Note that complete results of model evaluation that include other metrics can be found here: https://github.com/sergeypine/WeatherLearner/blob/capstone-model-experiments/model-prototyping/tf_benchmark2.txt)

NOTE: The above benchmarking script, instead of using all features and all locations uses only those features and locations that have been proven to benefit the prediction. For each of the 16 prediction targets, such optimal features and locations turned out to be different. See https://github.com/sergeypine/WeatherLearner/blob/capstone-model-experiments/model-prototyping/NOAA_feature_selector.py (this feature selection script tries features and locations one by one and evaluates whether such additions benefit performance) 

### Clarity Recall

| Prediction Interval | LINEAR | NN | DNN | CNN |
----------------------|--------|----|-----|-----|
| 6h | **.96** | .96 | .96 | .96 |
| 12h | **.81** | . 73 | . 77 | .71 |
| 18h | .74 | **. 81** | .73 | .77 |
| 24h | .64 | **.78** | . 64 | .76 |

Interestingly, for shorter time intervals `LINEAR` is the most optimal but a simple `Single Layer Neural Network` does better for the longer intervals.

We can see that for some models Recall actually increases as the forecast time increases, which at the first glance makes no sense. However, remember that we have optimized training specifically for `Recall`; such increases in `Recall` are more than compensated for by a drop in `Precision` (see full benchmarking results), resulting in far weaker models overall for longer forecast times. 

### Precipitation Recall

| Prediction Interval |  LINEAR | NN | DNN | CNN |
|--------------------|---------|----|-----|-----|
| 6h | **.81** | .81 | .81 | .81 |
| 12h | .44 | .58 | .50 | **.59** |
| 18h | .37 | .42 | .42 | **.53** |
| 24h | .26 | .45 | .29| **.46** | 

`Convolutional Neural Network` consistently gives the best results, except for the short term 6h forecast

### Temperature RMSE (degrees Farenheit)

| Prediction Interval |  LINEAR | NN | DNN | CNN |
|--------------------|---------|----|-----|-----|
| 6h | 3.39 | 3.03 | **2.97** | 3.04 |
| 12h | 5.61 | **4.61** | 4.96 | 4.81 | 
| 18h | 6.11 | 6.39 | **5.93** | 6.24 | 
| 24h | **6.71** | 8.83 | 9.65 | 9.41 |

Different types of `Neural Networks` work best for different intervals, except for the longest term 24h forecast. This is a rather puzzling conclusion.

### Wind Speed RMSE (miles per hour)

| Prediction Interval |  LINEAR | NN | DNN | CNN |
|--------------------|---------|----|-----|-----|
| 6h | 1.97 | **1.89** | 1.95 | 1.91 |
| 12h | 3.37 | **3.14** | 3.21 | 3.25 |
| 18h | 3.85 | **3.75** | 3.95 | 3.81 |
| 24h | **4.13** | 4.32 | 4.52 | 4.39 |

`Simple Neural Network` gives superior results except for the shortest term 6h forecast.